In [1]:
from Bio import SeqIO
from Bio import Seq
import tqdm
import pandas as pd

In [ ]:
# Extract Nematoda from RefSeq

In [114]:
species = []
gene = []
sequence = []

count_passed = 0
count_nucl = 0

seq_reader = SeqIO.parse('../../../mtDnaInvertebrate/Body/1Raw/mitochondrion.genomic.gbff', format='genbank')

for record in tqdm.tqdm(seq_reader):
    try:
        if 'Nematoda' in record.annotations['taxonomy']:

            for feature in record.features:
                if feature.type != 'CDS' or feature.strand != 1:
                    continue

                seq_str = str(feature.extract(record.seq))
                if len(seq_str) == 0:
                    count_passed += 1
                    continue

                sequence.append(seq_str)

                species.append(record.annotations['organism'].replace('/', ' ').replace('_', ' '))
                g = 'gene' if 'gene' in feature.qualifiers else 'product'
                gene.append(feature.qualifiers[g][0])
                count_nucl += 1

    except Seq.UndefinedSequenceError:
        continue

print(f'Пропущено последовательностей - {count_passed}')
print(f'Всего нуклеотидных последовательностей - {count_nucl}')

5041it [00:33, 131.94it/s]/home/emulciber/anaconda3/lib/python3.8/site-packages/Bio/GenBank/Scanner.py:1794: BiopythonParserWarning: Structured comment not parsed for NC_037498. Is it malformed?
  warnings.warn(
/home/emulciber/anaconda3/lib/python3.8/site-packages/Bio/GenBank/Scanner.py:1794: BiopythonParserWarning: Structured comment not parsed for NC_037499. Is it malformed?
  warnings.warn(
5309it [00:35, 186.36it/s]/home/emulciber/anaconda3/lib/python3.8/site-packages/Bio/GenBank/Scanner.py:1794: BiopythonParserWarning: Structured comment not parsed for NC_037500. Is it malformed?
  warnings.warn(
8965it [00:53, 264.01it/s]/home/emulciber/anaconda3/lib/python3.8/site-packages/Bio/GenBank/Scanner.py:1794: BiopythonParserWarning: Structured comment not parsed for NC_057213. Is it malformed?
  warnings.warn(
/home/emulciber/anaconda3/lib/python3.8/site-packages/Bio/GenBank/Scanner.py:1794: BiopythonParserWarning: Structured comment not parsed for NC_057212. Is it malformed?
  warning

Пропущено последовательностей - 0
Всего нуклеотидных последовательностей - 1983


In [ ]:
# Extract sequences from Bracht's file

In [115]:
species_b = []
gene_b = []
sequence_b = []

count_passed = 0
count_nucl = 0

seq_reader = SeqIO.parse('../../Body/1Raw/mtDNA - mephisto mtDNA project.gb', format='genbank')

for record in tqdm.tqdm(seq_reader):
    try:
        if 'Halicephalobus' in record.annotations['organism']:

            for feature in record.features:
                if feature.type != 'CDS' or feature.strand != 1:
                    continue

                seq_str = str(feature.extract(record.seq))
                if len(seq_str) == 0:
                    count_passed += 1
                    continue

                sequence_b.append(seq_str)

                species_b.append(record.annotations['organism'].replace('/', ' ').replace('_', ' '))
                g = 'gene' if 'gene' in feature.qualifiers else 'label'
                gene_b.append(feature.qualifiers[g][0])
                count_nucl += 1

    except Seq.UndefinedSequenceError:
        continue

print(f'Пропущено последовательностей - {count_passed}')
print(f'Всего нуклеотидных последовательностей - {count_nucl}')

38it [00:00, 520.03it/s]

Пропущено последовательностей - 0
Всего нуклеотидных последовательностей - 34


In [ ]:
# Include Plectus aquatilis - don't work, it's only 11 genes

In [116]:
'''
seq_reader = SeqIO.parse('../../Body/1Raw/plectus_aquatilis.gb', format='genbank')

for record in seq_reader:
    for feature in record.features:
        if feature.type != 'CDS' or feature.strand != 1:
            continue
            
        species_b.append(record.annotations['organism'].replace('/', ' ').replace('_', ' '))
        sequence_b.append(str(feature.extract(record.seq)))
        g = 'gene' if 'gene' in feature.qualifiers else 'label'
        gene_b.append(feature.qualifiers[g][0])
'''

In [117]:
species += species_b
gene += gene_b
sequence += sequence_b

In [118]:
derived_table = pd.DataFrame()
derived_table['Species'] = species
derived_table['Gene'] = gene
derived_table['Sequence'] = sequence

In [119]:
len(derived_table)

2028

In [122]:
derived_table['Gene'].unique()

array(['COX1', 'COX2', 'ND3', 'ND5', 'ND6', 'ND4L', 'ND1', 'ATP6', 'ND2',
       'CYTB', 'COX3', 'ND4', 'ATP8'], dtype=object)

In [121]:
derived_table['Gene'] = derived_table['Gene'].str.replace('ATP synthase F0 subunit ', 'ATP').str.replace('NADH dehydrogenase subunit ', 'ND')
derived_table['Gene'] = derived_table['Gene'].str.replace('cytochrome b', 'CYTB').str.replace('NAD', 'ND')
derived_table['Gene'] = derived_table['Gene'].str.replace('cytochrome c oxidase subunit III', 'COX3')
derived_table['Gene'] = derived_table['Gene'].str.replace('cytochrome c oxidase subunit II', 'COX2')
derived_table['Gene'] = derived_table['Gene'].str.replace('cytochrome c oxidase subunit I', 'COX1')
derived_table['Gene'] = derived_table['Gene'].str.replace('COIII', 'COX3')
derived_table['Gene'] = derived_table['Gene'].str.replace('COII', 'COX2')
derived_table['Gene'] = derived_table['Gene'].str.replace('COI', 'COX1')

In [123]:
genes = derived_table[derived_table['Species'] == 'Halicephalobus mephisto'].Gene.values

In [124]:
a = derived_table[derived_table.Gene.isin(genes)].groupby('Species').Gene.nunique()
a = a[a == 12]
a.shape

(148,)

In [126]:
preferable_df = derived_table[derived_table.Species.isin(a.index)]

In [131]:
preferable_df[preferable_df['Species'].str.contains('Plectus') == True]

,Species,Gene,Sequence


In [128]:
set(preferable_df.Gene.unique()).difference(genes)

set()

In [129]:
set(genes).difference(preferable_df.Gene.unique())

set()

In [130]:
preferable_df.to_csv('../../Body/2Derived/NematodaForIQTree.csv', index=False, sep=',')